In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fastai.transforms import TfmType
from fasterai.transforms import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(1)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [3]:
#NOTES:  Changed to 1/4 learning rate difference
BEDROOM = Path('data/lsun/bedroom')

proj_id = 'bedroom_attn3'
TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

c_lr=4e-4
c_lrs = np.array([c_lr,c_lr,c_lr])

g_lr=c_lr/4
g_lrs = np.array([g_lr])

x_noise=128
sn=True
self_attention=True
torch.backends.cudnn.benchmark=True

## Training

In [4]:
netG = DCGenerator(x_noise=x_noise, nf=64, scale=64, self_attention=self_attention, sn=sn).cuda()
#netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
#load_model(netG, gpath)

netD = DCCritic(nf=128, scale=16, self_attention=self_attention, sn=sn).cuda()
#netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

In [5]:
trainer = WGANTrainer(netD=netD, netG=netG, genloss_fns=[], sn=sn)
trainerVis = WganVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer', jupyter=False, visual_iters=200)

In [6]:
scheds=[]

#scheds.extend(WGANTrainSchedule.generate_schedules(szs=[32,32,32,32], bss=[256,256,256,256], path=BEDROOM, keep_pcts=[1.0,1.0,1.0,1.0], 
       # save_base_name=proj_id, c_lrs=c_lrs, g_lrs=g_lrs, lrs_unfreeze_factor=1.0, gen_freeze_tos=[0,0,0,0], x_noise=x_noise))

scheds.extend(WGANTrainSchedule.generate_schedules(szs=[64,64,64], bss=[64,64,64], path=BEDROOM, keep_pcts=[1.0,1.0,1.0], 
        save_base_name=proj_id, c_lrs=c_lrs, g_lrs=g_lrs, lrs_unfreeze_factor=1.0, gen_freeze_tos=[0,0,0], x_noise=x_noise))

#scheds.extend(WGANTrainSchedule.generate_schedules(szs=[128,128], bss=[32,32], path=BEDROOM, keep_pcts=[1.0,1.0], save_base_name=proj_id, 
        #c_lrs=c_lrs/4, g_lrs=g_lrs/4, lrs_unfreeze_factor=1.0, gen_freeze_tos=[0,0], x_noise=x_noise))

#scheds.extend(WGANTrainSchedule.generate_schedules(szs=[256], bss=[8], path=BEDROOM, keep_pcts=[1.0], save_base_name=proj_id, 
        #c_lrs=c_lrs/16, g_lrs=g_lrs/16, lrs_unfreeze_factor=1.0, gen_freeze_tos=[0], x_noise=x_noise))

In [ ]:
trainer.train(scheds=scheds)

 25%|██▌       | 11755/46918 [55:36<4:42:52,  2.07it/s]